# NuScenes 单目点云生成教程

本notebook完整复刻了NuScenes点云生成流程，将forward方法拆分为多个独立步骤，便于理解和调试。

## 功能概述

1. 从深度图生成点云
2. 支持多种稀疏度级别（Drop90, Drop80, Drop50, Drop25, full）
3. 支持天空过滤和深度一致性检查
4. 支持边界框裁剪
5. 使用depth_utils进行深度图预处理（插值到原始尺寸）

## 使用说明

1. 按顺序执行所有单元格
2. 在"配置参数"单元格中修改场景路径和参数
3. 每个步骤单元格可以独立检查和调试
4. 查看"反直觉检查"单元格以发现潜在问题


## 1. 环境配置

安装和导入所有必要的依赖包。


In [1]:
# 安装依赖（如果需要）
# !pip install numpy imageio opencv-python open3d rich torch

import os
import sys
import numpy as np
import imageio.v2 as imageio
import cv2
import open3d as o3d
from typing import Literal
from rich.console import Console
from copy import deepcopy

# 添加项目路径以导入模块
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.insert(0, project_root)

# 添加depth_utils路径
depth_utils_path = os.path.join(
    project_root, 
    'third_party/EVolSplat/preprocess/metric3d/mono/tools'
)
sys.path.insert(0, depth_utils_path)

# 初始化Console
CONSOLE = Console(width=120)

# 检查关键依赖
try:
    import torch
    CONSOLE.log("[green]✓ PyTorch installed[/green]")
except ImportError:
    CONSOLE.log("[red]✗ PyTorch not found. Please install: pip install torch[/red]")

try:
    from depth_utils import process_depth_for_use, load_depth_with_metadata
    CONSOLE.log("[green]✓ depth_utils imported successfully[/green]")
except ImportError as e:
    CONSOLE.log(f"[red]✗ Failed to import depth_utils: {e}[/red]")
    CONSOLE.log(f"[yellow]Make sure depth_utils.py is at: {depth_utils_path}[/yellow]")

CONSOLE.log("[green]Environment setup completed![/green]")


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


[16:28:51] ✓ PyTorch installed                                                                           ]8;id=961614;file:///tmp/ipykernel_22413/479480832.py\479480832.py]8;;\:]8;id=339290;file:///tmp/ipykernel_22413/479480832.py#31\31]8;;\

           ✓ depth_utils imported successfully                                                           ]8;id=651076;file:///tmp/ipykernel_22413/479480832.py\479480832.py]8;;\:]8;id=910516;file:///tmp/ipykernel_22413/479480832.py#37\37]8;;\

           Environment setup completed!                                                                  ]8;id=172765;file:///tmp/ipykernel_22413/479480832.py\479480832.py]8;;\:]8;id=705079;file:///tmp/ipykernel_22413/479480832.py#42\42]8;;\

## 2. 配置参数

修改以下参数以适配你的场景。


In [2]:
# ========== 场景配置 ==========
# 场景目录路径（例如: /path/to/trainval/000）
scene_dir = "/root/autodl-tmp/nuScenes/preprocess/trainval/000"

# ========== 相机配置 ==========
# 选择要使用的相机ID列表，例如 [0] 表示只使用前置摄像头，[0, 1, 2, 3, 4, 5] 表示使用所有相机
# 注意：必须与深度图中实际存在的相机ID匹配
chosen_cam_id = [0]  # 如果深度图只有000_0, 001_0等，则设置为[0]

# ========== 稀疏度配置 ==========
# 可选值: 'Drop90', 'Drop80', 'Drop50', 'Drop25', 'full'
sparsity = 'Drop50'

# ========== 帧范围配置 ==========
frame_start = 0  # 起始帧索引
num_frames = 20  # 要处理的帧数（None表示处理所有帧）

# ========== 功能开关 ==========
filter_sky = True  # 是否过滤天空区域
depth_consistency = True  # 是否进行深度一致性检查
use_bbx = True  # 是否使用边界框裁剪

# ========== 下采样配置 ==========
downscale = 2  # 点云生成时的下采样倍数

# ========== 保存配置 ==========
save_dir = None  # 保存目录名称（None则使用sparsity值）

# 显示配置
CONSOLE.log(f"[cyan]Configuration:[/cyan]")
CONSOLE.log(f"  Scene directory: {scene_dir}")
CONSOLE.log(f"  Chosen camera IDs: {chosen_cam_id}")
CONSOLE.log(f"  Sparsity: {sparsity}")
CONSOLE.log(f"  Frame range: {frame_start} to {frame_start + (num_frames if num_frames else 'all')}")
CONSOLE.log(f"  Filter sky: {filter_sky}")
CONSOLE.log(f"  Depth consistency: {depth_consistency}")
CONSOLE.log(f"  Use bounding box: {use_bbx}")
CONSOLE.log(f"  Downscale: {downscale}")


           Configuration:                                                                               ]8;id=134620;file:///tmp/ipykernel_22413/3050187033.py\3050187033.py]8;;\:]8;id=822960;file:///tmp/ipykernel_22413/3050187033.py#30\30]8;;\

             Scene directory: /root/autodl-tmp/nuScenes/preprocess/trainval/000                         ]8;id=367285;file:///tmp/ipykernel_22413/3050187033.py\3050187033.py]8;;\:]8;id=107195;file:///tmp/ipykernel_22413/3050187033.py#31\31]8;;\

             Chosen camera IDs: [0]                                                                     ]8;id=26038;file:///tmp/ipykernel_22413/3050187033.py\3050187033.py]8;;\:]8;id=44755;file:///tmp/ipykernel_22413/3050187033.py#32\32]8;;\

             Sparsity: Drop50                                                                           ]8;id=771698;file:///tmp/ipykernel_22413/3050187033.py\3050187033.py]8;;\:]8;id=644325;file:///tmp/ipykernel_22413/3050187033.py#33\33]8;;\

             Frame range: 0 to 20                                                                       ]8;id=103698;file:///tmp/ipykernel_22413/3050187033.py\3050187033.py]8;;\:]8;id=532976;file:///tmp/ipykernel_22413/3050187033.py#34\34]8;;\

             Filter sky: True                                                                           ]8;id=522408;file:///tmp/ipykernel_22413/3050187033.py\3050187033.py]8;;\:]8;id=515377;file:///tmp/ipykernel_22413/3050187033.py#35\35]8;;\

             Depth consistency: True                                                                    ]8;id=455083;file:///tmp/ipykernel_22413/3050187033.py\3050187033.py]8;;\:]8;id=777778;file:///tmp/ipykernel_22413/3050187033.py#36\36]8;;\

             Use bounding box: True                                                                     ]8;id=926208;file:///tmp/ipykernel_22413/3050187033.py\3050187033.py]8;;\:]8;id=734146;file:///tmp/ipykernel_22413/3050187033.py#37\37]8;;\

             Downscale: 2                                                                               ]8;id=764167;file:///tmp/ipykernel_22413/3050187033.py\3050187033.py]8;;\:]8;id=776961;file:///tmp/ipykernel_22413/3050187033.py#38\38]8;;\

## 3. NuScenes场景文件夹结构

### 目录结构

```
scene_dir/ (例如: /path/to/trainval/000)
├── images/                    # RGB图像目录
│   ├── 000_0.jpg (或 .png)   # 格式: {frame_idx:03d}_{cam_id}.jpg
│   ├── 000_1.jpg
│   ├── 001_0.jpg
│   └── ...
├── depth/                     # 深度图目录
│   ├── 000_0.npy             # 格式: {frame_idx:03d}_{cam_id}.npy
│   ├── 000_0_meta.npz        # 元数据文件（包含原始尺寸、内参等）
│   ├── 000_1.npy
│   ├── 000_1_meta.npz
│   └── ...
├── extrinsics/                # 相机外参（cam_to_world变换矩阵）
│   ├── 000_0.txt             # 格式: {frame_idx:03d}_{cam_id}.txt (4x4矩阵)
│   ├── 000_1.txt
│   └── ...
├── intrinsics/                # 相机内参（每个相机固定）
│   ├── 0.txt                 # 格式: {cam_id}.txt (fx, fy, cx, cy)
│   ├── 1.txt
│   └── ...
├── sky_masks/                 # 天空掩码（可选）
│   ├── 000_0.png             # 格式: {frame_idx:03d}_{cam_id}.png (0=sky, 255=non-sky)
│   └── ...
├── transforms.json           # 场景变换信息（可选）
└── [sparsity_dir]/           # 点云输出目录（如Drop50, Drop80等）
    └── 0.ply                 # 生成的点云文件
```

### 文件命名规则

- **图像**: `{frame_idx:03d}_{cam_id}.jpg` 或 `.png`
- **深度图**: `{frame_idx:03d}_{cam_id}.npy` (对应元数据: `{frame_idx:03d}_{cam_id}_meta.npz`)
- **外参**: `{frame_idx:03d}_{cam_id}.txt` (4x4矩阵，每行一个值)
- **内参**: `{cam_id}.txt` (fx, fy, cx, cy，每行一个值)

### 重要说明

1. **深度图尺寸**: 深度图可能不是原始图像尺寸（模型输出去除了padding），需要使用`depth_utils.process_depth_for_use()`进行插值
2. **元数据文件**: 每个深度图都有对应的`_meta.npz`文件，包含原始尺寸、内参等信息
3. **坐标系统**: 外参是cam_to_world变换矩阵，已经对齐到第一帧第一相机


## 4. 辅助函数

定义读取图像尺寸和位姿/内参的函数。


In [3]:
def get_image_dimensions(scene_dir: str):
    """
    从场景目录获取图像尺寸。
    
    Args:
        scene_dir: 场景目录路径
        
    Returns:
        (H, W): 图像高度和宽度
    """
    images_dir = os.path.join(scene_dir, 'images')
    if not os.path.exists(images_dir):
        raise ValueError(f"Images directory not found: {images_dir}")
    
    image_files = [f for f in os.listdir(images_dir) 
                   if f.endswith('.jpg') or f.endswith('.png')]
    if len(image_files) == 0:
        raise ValueError(f"No image files found in {images_dir}")
    
    # 读取第一张图像获取尺寸
    first_image_path = os.path.join(images_dir, sorted(image_files)[0])
    img = imageio.imread(first_image_path)
    H, W = img.shape[0], img.shape[1]
    
    return H, W


def read_poses_intrinsics_direct(scene_dir: str, frame_start: int = 0, num_frames: int = None):
    """
    直接从场景目录读取poses和intrinsics，按照NuScenes场景文件夹的预定结构。
    
    Args:
        scene_dir: 场景目录路径
        frame_start: 起始帧索引（默认0）
        num_frames: 要读取的帧数（None表示读取所有帧）
        
    Returns:
        poses: Array of poses (num_frames * num_cameras, 4, 4)
        intrinsics: Array of intrinsics (num_frames * num_cameras, 4, 4)
        info: Tuple (H, W) - 图像尺寸
    """
    extrinsics_dir = os.path.join(scene_dir, 'extrinsics')
    intrinsics_dir = os.path.join(scene_dir, 'intrinsics')
    images_dir = os.path.join(scene_dir, 'images')
    
    if not os.path.exists(extrinsics_dir):
        raise ValueError(f"Extrinsics directory not found: {extrinsics_dir}")
    if not os.path.exists(intrinsics_dir):
        raise ValueError(f"Intrinsics directory not found: {intrinsics_dir}")
    if not os.path.exists(images_dir):
        raise ValueError(f"Images directory not found: {images_dir}")
    
    # 获取所有帧索引和相机ID
    extrinsic_files = sorted([f for f in os.listdir(extrinsics_dir) if f.endswith('.txt')])
    frame_cam_pairs = []
    for f in extrinsic_files:
        try:
            parts = f.replace('.txt', '').split('_')
            if len(parts) >= 2:
                frame_idx = int(parts[0])
                cam_id = int(parts[1])
                frame_cam_pairs.append((frame_idx, cam_id))
        except (ValueError, IndexError):
            continue
    
    # 过滤帧范围
    if num_frames is not None:
        frame_end = frame_start + num_frames
        frame_cam_pairs = [(f, c) for f, c in frame_cam_pairs if frame_start <= f < frame_end]
    else:
        frame_cam_pairs = [(f, c) for f, c in frame_cam_pairs if f >= frame_start]
    
    # 按frame_idx和cam_id排序
    frame_cam_pairs.sort(key=lambda x: (x[0], x[1]))
    
    # 加载第一帧第一相机的pose用于对齐
    camera_front_start = None
    first_frame_cam = None
    for frame_idx, cam_id in frame_cam_pairs:
        if cam_id == 0:  # 使用相机0作为参考
            first_extrinsic_file = os.path.join(extrinsics_dir, f'{frame_idx:03d}_{cam_id}.txt')
            if os.path.exists(first_extrinsic_file):
                camera_front_start = np.loadtxt(first_extrinsic_file)
                first_frame_cam = (frame_idx, cam_id)
                break
    
    if camera_front_start is None:
        # Fallback: 使用第一个可用的extrinsic
        if len(frame_cam_pairs) > 0:
            first_frame_cam = frame_cam_pairs[0]
            first_extrinsic_file = os.path.join(extrinsics_dir, f'{first_frame_cam[0]:03d}_{first_frame_cam[1]}.txt')
            if os.path.exists(first_extrinsic_file):
                camera_front_start = np.loadtxt(first_extrinsic_file)
                CONSOLE.log(f"Warning: Using frame {first_frame_cam[0]} camera {first_frame_cam[1]} for alignment")
    
    # 读取所有poses和intrinsics
    poses = []
    intrinsics = []
    
    # 获取相机ID列表
    cam_ids = sorted(set(cam_id for _, cam_id in frame_cam_pairs))
    
    # 读取每个相机的内参（固定值）
    cam_intrinsics_dict = {}
    for cam_id in cam_ids:
        intrinsic_file = os.path.join(intrinsics_dir, f'{cam_id}.txt')
        if os.path.exists(intrinsic_file):
            intrinsic_data = np.loadtxt(intrinsic_file)
            fx, fy, cx, cy = intrinsic_data[0], intrinsic_data[1], intrinsic_data[2], intrinsic_data[3]
            # 转换为3x3矩阵（与nuscenes_sourceloader.py保持一致）
            cam_intrinsic = np.array([[fx, 0, cx], [0, fy, cy], [0, 0, 1]])
            cam_intrinsics_dict[cam_id] = cam_intrinsic
        else:
            CONSOLE.log(f"Warning: Intrinsic file not found: {intrinsic_file}")
    
    # 读取每个frame_cam对的pose
    for frame_idx, cam_id in frame_cam_pairs:
        extrinsic_file = os.path.join(extrinsics_dir, f'{frame_idx:03d}_{cam_id}.txt')
        if not os.path.exists(extrinsic_file):
            CONSOLE.log(f"Warning: Extrinsic file not found: {extrinsic_file}, skipping")
            continue
        
        # 加载外参（cam_to_world）
        cam2world = np.loadtxt(extrinsic_file)
        
        # 对齐到第一帧第一相机（如果可用）
        if camera_front_start is not None:
            cam2world = np.linalg.inv(camera_front_start) @ cam2world
        
        # OPENCV2DATASET是单位矩阵，不需要转换
        OPENCV2DATASET = np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]])
        cam2world = cam2world @ OPENCV2DATASET
        
        poses.append(cam2world)
        
        # 添加对应的内参
        if cam_id in cam_intrinsics_dict:
            intrinsics.append(cam_intrinsics_dict[cam_id])
        else:
            # 如果内参不存在，使用单位矩阵（3x3）
            intrinsics.append(np.eye(3))
    
    # 获取图像尺寸
    H, W = get_image_dimensions(scene_dir)
    
    return np.array(poses), np.array(intrinsics), (H, W)

CONSOLE.log("[green]Helper functions defined![/green]")


           Helper functions defined!                                                                   ]8;id=357780;file:///tmp/ipykernel_22413/2343214192.py\2343214192.py]8;;\:]8;id=306290;file:///tmp/ipykernel_22413/2343214192.py#148\148]8;;\

In [4]:
# 导入depth_utils
try:
    from depth_utils import process_depth_for_use, load_depth_with_metadata
    CONSOLE.log("[green]✓ depth_utils imported successfully[/green]")
    
    # 示例：如何加载和预处理深度图
    # depth_file = os.path.join(scene_dir, 'depth', '000_0.npy')
    # depth, metadata = process_depth_for_use(depth_file, target_shape=(H, W))
    # print(f"Depth shape: {depth.shape}")
    # print(f"Original shape: {metadata['ori_shape']}")
    
except ImportError as e:
    CONSOLE.log(f"[red]✗ Failed to import depth_utils: {e}[/red]")
    CONSOLE.log("[yellow]Make sure depth_utils.py is in the correct path[/yellow]")


           ✓ depth_utils imported successfully                                                           ]8;id=682517;file:///tmp/ipykernel_22413/3421549315.py\3421549315.py]8;;\:]8;id=954370;file:///tmp/ipykernel_22413/3421549315.py#4\4]8;;\

## 6. NuScenesPCDGenerator类定义

定义点云生成器的辅助方法（不包含forward方法，forward将被拆分为多个步骤）。


In [5]:
# Default bounding box for nuScenes (similar to Waymo, can be customized)
X_MIN, X_MAX = -20, 20
Y_MIN, Y_MAX = -20, 4.8
Z_MIN, Z_MAX = -20, 70

class NuScenesPCDGenerator():
    """Point cloud generator for nuScenes dataset."""
    sparsity: Literal['Drop90','Drop50',"Drop80","Drop25","full"] = "full"
    use_bbx: bool = True

    def __init__(self, spars='full', save_dir="Drop50", frame_start=0, 
                 filer_sky=True, depth_cosistency=True) -> None:
        self.sparsity = spars
        self.save_dir = save_dir
        self.dir_name = None
        self.frame_start = frame_start
        self.filter_sky = filer_sky
        self.depth_cosistency = depth_cosistency

    def get_bbx(self):
        """Get bounding box for point cloud filtering."""
        return np.array([X_MIN, Y_MIN, Z_MIN]), np.array([X_MAX, Y_MAX, Z_MAX])

    def crop_pointcloud(self, bbx_min, bbx_max, points, color):
        """Crop point cloud to bounding box."""
        mask = (points[:, 0] > bbx_min[0]) & (points[:, 0] < bbx_max[0]) & \
            (points[:, 1] > bbx_min[1]) & (points[:, 1] < bbx_max[1]) & \
            (points[:, 2] > bbx_min[2]) & (points[:, 2] < bbx_max[2] + 50)  # Extended Z for background
        
        return points[mask], color[mask]
    
    def split_pointcloud(self, bbx_min, bbx_max, points, color):
        """Split point cloud into inside and outside bounding box."""
        mask = (points[:, 0] > bbx_min[0]) & (points[:, 0] < bbx_max[0]) & \
            (points[:, 1] > bbx_min[1]) & (points[:, 1] < bbx_max[1]) & \
            (points[:, 2] > bbx_min[2]) & (points[:, 2] < bbx_max[2])
        
        inside_pnt, inside_rgb = points[mask], color[mask]
        outside_pnt, outside_rgb = points[~mask], color[~mask]
        return inside_pnt, inside_rgb, outside_pnt, outside_rgb

    def depth_cosistency_check(self, depth_files, H, W):
        """Check depth consistency between consecutive frames."""
        depth_masks = []
        print("Depth Consistency Check!")
        
        for i, file_name in enumerate(depth_files):
            depth_file = os.path.join(self.depth_dir, file_name)
            try:
                # 使用depth_utils加载深度图
                depth, metadata = process_depth_for_use(depth_file, target_shape=(H, W))
            except Exception as e:
                CONSOLE.log(f"Warning: Failed to load depth {file_name}: {e}")
                depth_masks.append(np.ones((H, W)))
                continue

            # Assume the first depth frame is correct
            if i == 0:
                last_depth = deepcopy(depth)
                depth_masks.append(np.ones((H, W)))
                continue

            try:
                c2w = self.c2w[i]
                last_c2w = self.c2w[i-1]
                K = self.intri[i]
            except IndexError as e:
                depth_masks.append(np.ones((H, W)))
                continue

            # Unproject pointcloud
            x = np.arange(0, depth.shape[1])
            y = np.arange(0, depth.shape[0])
            xx, yy = np.meshgrid(x, y)
            pixels = np.vstack((xx.ravel(), yy.ravel())).T.reshape(-1, 2)

            # Unproject depth to pointcloud
            cx, cy = K[0, 2], K[1, 2]
            fx, fy = K[0, 0], K[1, 1]
            
            x_cam = (pixels[..., 0] - cx) * depth.reshape(-1) / fx
            y_cam = (pixels[..., 1] - cy) * depth.reshape(-1) / fy
            z_cam = depth.reshape(-1)
            coordinates = np.stack([x_cam, y_cam, z_cam], axis=1)

            depth_mask = self.depth_projection_check(
                coordinates=coordinates, pixels=pixels, 
                last_c2w=last_c2w, c2w=c2w, 
                last_depth=last_depth, depth=depth, K=K
            )
            depth_masks.append(depth_mask)

            # Update status
            last_depth = deepcopy(depth)

        return depth_masks

    def depth_projection_check(self, coordinates, pixels, last_c2w, c2w, last_depth, depth, K):
        """Check depth consistency by projecting current points to previous frame."""
        H, W = last_depth.shape[:2]
        cx, cy = K[0, 2], K[1, 2]
        fx, fy = K[0, 0], K[1, 1]

        trans_mat = np.dot(np.linalg.inv(last_c2w), c2w)
        coordinates_homo = np.column_stack((coordinates.reshape(-1, 3), np.ones(len(coordinates))))
        last_coordinates = np.dot(trans_mat, coordinates_homo.T).T
        
        # Project to previous frame
        last_x = (fx * last_coordinates[:, 0] + cx * last_coordinates[:, 2]) / last_coordinates[:, 2]
        last_y = (fy * last_coordinates[:, 1] + cy * last_coordinates[:, 2]) / last_coordinates[:, 2]
        last_pixels = np.vstack((last_x, last_y)).T.reshape(-1, 2).astype(np.int32)

        # Swap pixel coordinates (row, col) <-> (x, y)
        pixels_swapped = pixels.copy()
        pixels_swapped[:, [0, 1]] = pixels_swapped[:, [1, 0]]
        last_pixels_swapped = last_pixels.copy()
        last_pixels_swapped[:, [0, 1]] = last_pixels_swapped[:, [1, 0]]

        depth_mask = np.ones(depth.shape[0] * depth.shape[1])

        # Reprojection location must be in image plane [0,H] [0,W]
        valid_mask_00 = np.logical_and((last_pixels_swapped[:, 0] < H), (last_pixels_swapped[:, 1] < W))
        valid_mask_01 = np.logical_and((last_pixels_swapped[:, 0] > 0), (last_pixels_swapped[:, 1] > 0))
        valid_mask = np.logical_and(valid_mask_00, valid_mask_01)

        depth_diff = np.abs(
            depth[pixels_swapped[valid_mask, 0], pixels_swapped[valid_mask, 1]] - 
            last_depth[last_pixels_swapped[valid_mask, 0], last_pixels_swapped[valid_mask, 1]]
        )
        depth_mask[valid_mask] = np.where(depth_diff < depth_diff.mean(), 1, 0)
        depth_mask = depth_mask.reshape(*depth.shape)

        return depth_mask

CONSOLE.log("[green]NuScenesPCDGenerator class defined![/green]")


           NuScenesPCDGenerator class defined!                                                         ]8;id=261615;file:///tmp/ipykernel_22413/3590799288.py\3590799288.py]8;;\:]8;id=119042;file:///tmp/ipykernel_22413/3590799288.py#135\135]8;;\

In [6]:
# 验证场景目录
if not os.path.exists(scene_dir):
    raise ValueError(f"Scene directory not found: {scene_dir}")

# 检查必要的目录和文件
required_dirs = ['images', 'extrinsics', 'intrinsics', 'depth']
for dir_name in required_dirs:
    dir_path = os.path.join(scene_dir, dir_name)
    if not os.path.exists(dir_path):
        raise ValueError(f"Required directory not found: {dir_path}")
    CONSOLE.log(f"[green]✓ Found {dir_name}/[/green]")

# 检查天空掩码目录（如果启用天空过滤）
if filter_sky:
    sky_masks_dir = os.path.join(scene_dir, 'sky_masks')
    if not os.path.exists(sky_masks_dir):
        CONSOLE.log(f"[yellow]Warning: Sky masks directory not found: {sky_masks_dir}[/yellow]")
        CONSOLE.log("[yellow]Sky filtering will be disabled. Run gen_nuscenes_depth_mask.py --gen_sky_mask first.[/yellow]")
        filter_sky = False
    else:
        CONSOLE.log(f"[green]✓ Found sky_masks/[/green]")

# 验证深度文件存在
depth_dir = os.path.join(scene_dir, 'depth')
all_depth_files = [f for f in os.listdir(depth_dir) if f.endswith('.npy') and not f.endswith('_meta.npz')]
if len(all_depth_files) == 0:
    raise ValueError(f"No depth files found in {depth_dir}")
CONSOLE.log(f"[green]✓ Found {len(all_depth_files)} total depth files[/green]")

# 检查深度文件中的相机ID
depth_cam_ids = set()
for f in all_depth_files:
    try:
        parts = f.replace('.npy', '').split('_')
        if len(parts) >= 2:
            depth_cam_ids.add(int(parts[1]))
    except:
        continue

CONSOLE.log(f"[cyan]Camera ID analysis:[/cyan]")
CONSOLE.log(f"  Camera IDs found in depth files: {sorted(depth_cam_ids)}")
CONSOLE.log(f"  Chosen camera IDs: {chosen_cam_id}")

# 检查chosen_cam_id是否在深度文件中存在
missing_cams = set(chosen_cam_id) - depth_cam_ids
if missing_cams:
    CONSOLE.log(f"[yellow]⚠ Warning: Some chosen_cam_id ({missing_cams}) not found in depth files[/yellow]")
    CONSOLE.log(f"[yellow]  Available camera IDs: {sorted(depth_cam_ids)}[/yellow]")

# 检查是否有深度文件使用了不在chosen_cam_id中的相机
unexpected_cams = depth_cam_ids - set(chosen_cam_id)
if unexpected_cams:
    CONSOLE.log(f"[cyan]  Note: Depth files also contain cameras {sorted(unexpected_cams)} (will be filtered)[/cyan]")

# 检查深度图元数据文件
meta_files = [f for f in os.listdir(depth_dir) if f.endswith('_meta.npz')]
CONSOLE.log(f"[green]✓ Found {len(meta_files)} metadata files[/green]")
if len(meta_files) < len(all_depth_files):
    CONSOLE.log(f"[yellow]Warning: Some depth files may be missing metadata files[/yellow]")

CONSOLE.log("[green]Data validation completed![/green]")


           ✓ Found images/                                                                              ]8;id=64232;file:///tmp/ipykernel_22413/2378628925.py\2378628925.py]8;;\:]8;id=775314;file:///tmp/ipykernel_22413/2378628925.py#11\11]8;;\

           ✓ Found extrinsics/                                                                          ]8;id=34261;file:///tmp/ipykernel_22413/2378628925.py\2378628925.py]8;;\:]8;id=124817;file:///tmp/ipykernel_22413/2378628925.py#11\11]8;;\

           ✓ Found intrinsics/                                                                          ]8;id=739205;file:///tmp/ipykernel_22413/2378628925.py\2378628925.py]8;;\:]8;id=860431;file:///tmp/ipykernel_22413/2378628925.py#11\11]8;;\

           ✓ Found depth/                                                                               ]8;id=39588;file:///tmp/ipykernel_22413/2378628925.py\2378628925.py]8;;\:]8;id=308813;file:///tmp/ipykernel_22413/2378628925.py#11\11]8;;\

           ✓ Found sky_masks/                                                                           ]8;id=827604;file:///tmp/ipykernel_22413/2378628925.py\2378628925.py]8;;\:]8;id=636680;file:///tmp/ipykernel_22413/2378628925.py#21\21]8;;\

...

           Camera ID analysis:                                                                          ]8;id=789931;file:///tmp/ipykernel_22413/2378628925.py\2378628925.py]8;;\:]8;id=94585;file:///tmp/ipykernel_22413/2378628925.py#40\40]8;;\

             Camera IDs found in depth files: [0]                                                       ]8;id=767542;file:///tmp/ipykernel_22413/2378628925.py\2378628925.py]8;;\:]8;id=399584;file:///tmp/ipykernel_22413/2378628925.py#41\41]8;;\

             Chosen camera IDs: [0]                                                                     ]8;id=57533;file:///tmp/ipykernel_22413/2378628925.py\2378628925.py]8;;\:]8;id=554949;file:///tmp/ipykernel_22413/2378628925.py#42\42]8;;\

[16:28:52] ✓ Found 196 metadata files                                                                   ]8;id=821321;file:///tmp/ipykernel_22413/2378628925.py\2378628925.py]8;;\:]8;id=312459;file:///tmp/ipykernel_22413/2378628925.py#57\57]8;;\

           Data validation completed!                                                                   ]8;id=93911;file:///tmp/ipykernel_22413/2378628925.py\2378628925.py]8;;\:]8;id=425961;file:///tmp/ipykernel_22413/2378628925.py#61\61]8;;\

## 步骤1: 初始化与目录设置

设置目录路径、获取图像尺寸、初始化点云生成器。


In [7]:
# 初始化：只读取图像尺寸和第一帧第一相机的pose用于对齐
# 不再预先读取所有poses/intrinsics，改为根据深度文件直接读取
CONSOLE.log(f"Initializing scene: {scene_dir}")

# 获取图像尺寸
H, W = get_image_dimensions(scene_dir)

# 加载第一帧第一相机的pose用于对齐（参考坐标系）
extrinsics_dir = os.path.join(scene_dir, 'extrinsics')
camera_front_start = None
if os.path.exists(extrinsics_dir):
    # 尝试找到第一帧第一相机（cam_id=0）的pose
    first_extrinsic_file = os.path.join(extrinsics_dir, f"{frame_start:03d}_0.txt")
    if os.path.exists(first_extrinsic_file):
        camera_front_start = np.loadtxt(first_extrinsic_file)
        CONSOLE.log(f"[green]Loaded reference pose from {first_extrinsic_file}[/green]")
    else:
        # Fallback: 查找第一个可用的extrinsic文件
        extrinsic_files = sorted([f for f in os.listdir(extrinsics_dir) if f.endswith('.txt')])
        if len(extrinsic_files) > 0:
            first_extrinsic_file = os.path.join(extrinsics_dir, extrinsic_files[0])
            camera_front_start = np.loadtxt(first_extrinsic_file)
            CONSOLE.log(f"[yellow]Warning: Using {extrinsic_files[0]} as reference pose[/yellow]")
else:
    CONSOLE.log(f"[yellow]Warning: Extrinsics directory not found: {extrinsics_dir}[/yellow]")

# 创建点云生成器
save_dir_name = save_dir if save_dir is not None else sparsity
pcd_generator = NuScenesPCDGenerator(
    spars=sparsity,
    save_dir=save_dir_name,
    frame_start=frame_start,
    filer_sky=filter_sky,
    depth_cosistency=depth_consistency
)
pcd_generator.use_bbx = use_bbx

# 设置目录路径
pcd_generator.dir_name = scene_dir
pcd_generator.depth_dir = os.path.join(scene_dir, 'depth')
pcd_generator.H, pcd_generator.W = H, W

# 保存参考pose用于后续对齐
pcd_generator.camera_front_start = camera_front_start

CONSOLE.log("[green]Step 1: Initialization completed![/green]")
CONSOLE.log(f"  Image dimensions: H={H}, W={W}")
CONSOLE.log(f"  Chosen camera IDs: {chosen_cam_id}")
CONSOLE.log(f"  Reference pose loaded: {camera_front_start is not None}")
CONSOLE.log(f"[cyan]Note: Poses and intrinsics will be loaded directly from depth files in Step 5[/cyan]")


           Initializing scene: /root/autodl-tmp/nuScenes/preprocess/trainval/000                         ]8;id=221534;file:///tmp/ipykernel_22413/3720287474.py\3720287474.py]8;;\:]8;id=350792;file:///tmp/ipykernel_22413/3720287474.py#3\3]8;;\

           Loaded reference pose from                                                                   ]8;id=453844;file:///tmp/ipykernel_22413/3720287474.py\3720287474.py]8;;\:]8;id=659909;file:///tmp/ipykernel_22413/3720287474.py#16\16]8;;\
           /root/autodl-tmp/nuScenes/preprocess/trainval/000/extrinsics/000_0.txt                                       

           Step 1: Initialization completed!                                                            ]8;id=471879;file:///tmp/ipykernel_22413/3720287474.py\3720287474.py]8;;\:]8;id=732111;file:///tmp/ipykernel_22413/3720287474.py#46\46]8;;\

             Image dimensions: H=900, W=1600                                                            ]8;id=368834;file:///tmp/ipykernel_22413/3720287474.py\3720287474.py]8;;\:]8;id=83805;file:///tmp/ipykernel_22413/3720287474.py#47\47]8;;\

             Chosen camera IDs: [0]                                                                     ]8;id=286154;file:///tmp/ipykernel_22413/3720287474.py\3720287474.py]8;;\:]8;id=899223;file:///tmp/ipykernel_22413/3720287474.py#48\48]8;;\

             Reference pose loaded: True                                                                ]8;id=499026;file:///tmp/ipykernel_22413/3720287474.py\3720287474.py]8;;\:]8;id=944297;file:///tmp/ipykernel_22413/3720287474.py#49\49]8;;\

           Note: Poses and intrinsics will be loaded directly from depth files in Step 5                ]8;id=635395;file:///tmp/ipykernel_22413/3720287474.py\3720287474.py]8;;\:]8;id=136257;file:///tmp/ipykernel_22413/3720287474.py#50\50]8;;\

## 步骤2: 加载和分组深度文件

加载所有深度文件并按帧索引分组，为稀疏度过滤做准备。


In [8]:
# 加载所有深度文件（排除元数据文件）
all_depth_files = sorted([
    f for f in os.listdir(pcd_generator.depth_dir) 
    if f.endswith('.npy') and not f.endswith('_meta.npz')
])

CONSOLE.log(f"Found {len(all_depth_files)} total depth files")

# 根据chosen_cam_id过滤深度文件
# 深度文件格式: {frame_idx:03d}_{cam_id}.npy
filtered_depth_files = []
for file_name in all_depth_files:
    try:
        # 解析相机ID: {frame_idx:03d}_{cam_id}.npy
        parts = file_name.replace('.npy', '').split('_')
        if len(parts) >= 2:
            cam_id = int(parts[1])
            if cam_id in chosen_cam_id:
                filtered_depth_files.append(file_name)
    except (ValueError, IndexError):
        continue

CONSOLE.log(f"[cyan]Camera filtering:[/cyan]")
CONSOLE.log(f"  Chosen camera IDs: {chosen_cam_id}")
CONSOLE.log(f"  Filtered depth files: {len(filtered_depth_files)} (from {len(all_depth_files)} total)")

# 按帧索引分组（只包含chosen_cam_id中的相机）
frame_groups = {}
for file_name in filtered_depth_files:
    try:
        # 解析帧索引: {frame_idx:03d}_{cam_id}.npy
        frame_idx = int(file_name.split('_')[0])
        if frame_idx not in frame_groups:
            frame_groups[frame_idx] = []
        frame_groups[frame_idx].append(file_name)
    except (ValueError, IndexError) as e:
        CONSOLE.log(f"Warning: Cannot parse filename {file_name}, skipping")
        continue

# 显示分组统计
sorted_frame_indices = sorted(frame_groups.keys())
CONSOLE.log(f"[cyan]Frame grouping statistics:[/cyan]")
CONSOLE.log(f"  Total frames: {len(sorted_frame_indices)}")
if len(sorted_frame_indices) > 0:
    CONSOLE.log(f"  Frame range: {sorted_frame_indices[0]} to {sorted_frame_indices[-1]}")
    cameras_per_frame = len(frame_groups[sorted_frame_indices[0]])
    CONSOLE.log(f"  Cameras per frame: {cameras_per_frame} (should match chosen_cam_id: {len(chosen_cam_id)})")

CONSOLE.log("[green]Step 2: Depth file loading and grouping completed![/green]")


           Found 196 total depth files                                                                    ]8;id=682760;file:///tmp/ipykernel_22413/278330606.py\278330606.py]8;;\:]8;id=280138;file:///tmp/ipykernel_22413/278330606.py#7\7]8;;\

           Camera filtering:                                                                             ]8;id=794615;file:///tmp/ipykernel_22413/278330606.py\278330606.py]8;;\:]8;id=463252;file:///tmp/ipykernel_22413/278330606.py#23\23]8;;\

             Chosen camera IDs: [0]                                                                      ]8;id=25376;file:///tmp/ipykernel_22413/278330606.py\278330606.py]8;;\:]8;id=431250;file:///tmp/ipykernel_22413/278330606.py#24\24]8;;\

             Filtered depth files: 196 (from 196 total)                                                  ]8;id=405569;file:///tmp/ipykernel_22413/278330606.py\278330606.py]8;;\:]8;id=128888;file:///tmp/ipykernel_22413/278330606.py#25\25]8;;\

           Frame grouping statistics:                                                                    ]8;id=598075;file:///tmp/ipykernel_22413/278330606.py\278330606.py]8;;\:]8;id=228850;file:///tmp/ipykernel_22413/278330606.py#42\42]8;;\

             Total frames: 196                                                                           ]8;id=642313;file:///tmp/ipykernel_22413/278330606.py\278330606.py]8;;\:]8;id=81843;file:///tmp/ipykernel_22413/278330606.py#43\43]8;;\

             Frame range: 0 to 195                                                                       ]8;id=148586;file:///tmp/ipykernel_22413/278330606.py\278330606.py]8;;\:]8;id=351602;file:///tmp/ipykernel_22413/278330606.py#45\45]8;;\

             Cameras per frame: 1 (should match chosen_cam_id: 1)                                        ]8;id=938058;file:///tmp/ipykernel_22413/278330606.py\278330606.py]8;;\:]8;id=312396;file:///tmp/ipykernel_22413/278330606.py#47\47]8;;\

           Step 2: Depth file loading and grouping completed!                                            ]8;id=879926;file:///tmp/ipykernel_22413/278330606.py\278330606.py]8;;\:]8;id=821214;file:///tmp/ipykernel_22413/278330606.py#49\49]8;;\

## 步骤3: 稀疏度过滤

根据稀疏度级别过滤帧，减少点云密度。


In [9]:
# 根据稀疏度级别过滤帧
selected_frames = []
for frame_pos, frame_idx in enumerate(sorted_frame_indices):
    # 应用稀疏度过滤
    if sparsity == "Drop50":
        if frame_pos % 4 == 2 or frame_pos % 4 == 3:
            continue  # 保留50%的帧
    elif sparsity == 'Drop80':
        if frame_pos % 5 != 0:  # 保留20%的帧
            continue 
    elif sparsity == 'Drop25':
        if frame_pos % 4 == 2:  # 保留75%的帧
            continue 
    elif sparsity == 'Drop90':
        if frame_pos % 10 != 0:  # 保留10%的帧
            continue
    # 'full' 不进行过滤
    
    selected_frames.append(frame_idx)

# 收集选中帧的深度文件（按frame_idx和cam_id排序）
# 注意：frame_groups中已经只包含chosen_cam_id中的相机
depth_files = []
for frame_idx in selected_frames:
    if frame_idx in frame_groups:
        # 确保按frame_idx和cam_id排序
        depth_files.extend(sorted(frame_groups[frame_idx]))

CONSOLE.log(f"[cyan]Sparsity filtering results:[/cyan]")
CONSOLE.log(f"  Sparsity level: {sparsity}")
CONSOLE.log(f"  Total frames: {len(sorted_frame_indices)}")
CONSOLE.log(f"  Selected frames: {len(selected_frames)}")
CONSOLE.log(f"  Selected depth files: {len(depth_files)}")
CONSOLE.log(f"  Expected files per frame: {len(chosen_cam_id)} cameras")

CONSOLE.log("[green]Step 3: Sparsity filtering completed![/green]")


           Sparsity filtering results:                                                                  ]8;id=406046;file:///tmp/ipykernel_22413/1391159818.py\1391159818.py]8;;\:]8;id=618390;file:///tmp/ipykernel_22413/1391159818.py#29\29]8;;\

             Sparsity level: Drop50                                                                     ]8;id=499600;file:///tmp/ipykernel_22413/1391159818.py\1391159818.py]8;;\:]8;id=351850;file:///tmp/ipykernel_22413/1391159818.py#30\30]8;;\

             Total frames: 196                                                                          ]8;id=410319;file:///tmp/ipykernel_22413/1391159818.py\1391159818.py]8;;\:]8;id=83001;file:///tmp/ipykernel_22413/1391159818.py#31\31]8;;\

             Selected frames: 98                                                                        ]8;id=595955;file:///tmp/ipykernel_22413/1391159818.py\1391159818.py]8;;\:]8;id=288369;file:///tmp/ipykernel_22413/1391159818.py#32\32]8;;\

             Selected depth files: 98                                                                   ]8;id=167951;file:///tmp/ipykernel_22413/1391159818.py\1391159818.py]8;;\:]8;id=117863;file:///tmp/ipykernel_22413/1391159818.py#33\33]8;;\

             Expected files per frame: 1 cameras                                                        ]8;id=158551;file:///tmp/ipykernel_22413/1391159818.py\1391159818.py]8;;\:]8;id=452470;file:///tmp/ipykernel_22413/1391159818.py#34\34]8;;\

           Step 3: Sparsity filtering completed!                                                        ]8;id=292596;file:///tmp/ipykernel_22413/1391159818.py\1391159818.py]8;;\:]8;id=200963;file:///tmp/ipykernel_22413/1391159818.py#36\36]8;;\

## 步骤4: 构建Pose映射

构建(frame_idx, cam_id)到pose/intrinsic索引的映射，用于匹配深度文件与poses。


In [10]:
# 步骤4已删除：不再需要构建pose_map
# 重构后的流程：根据深度文件直接读取对应的外参和内参文件
# 这样可以避免索引不匹配的问题，特别是当深度图只包含部分相机时
CONSOLE.log("[green]Step 4: Skipped (pose mapping no longer needed)[/green]")
CONSOLE.log("[cyan]Note: Poses and intrinsics will be loaded directly from depth files in Step 5[/cyan]")
CONSOLE.log("[cyan]This ensures correct matching even when depth files only contain a subset of cameras[/cyan]")


           Step 4: Skipped (pose mapping no longer needed)                                                ]8;id=871972;file:///tmp/ipykernel_22413/873654463.py\873654463.py]8;;\:]8;id=770825;file:///tmp/ipykernel_22413/873654463.py#4\4]8;;\

           Note: Poses and intrinsics will be loaded directly from depth files in Step 5                  ]8;id=772377;file:///tmp/ipykernel_22413/873654463.py\873654463.py]8;;\:]8;id=283224;file:///tmp/ipykernel_22413/873654463.py#5\5]8;;\

           This ensures correct matching even when depth files only contain a subset of cameras           ]8;id=299342;file:///tmp/ipykernel_22413/873654463.py\873654463.py]8;;\:]8;id=187361;file:///tmp/ipykernel_22413/873654463.py#6\6]8;;\

## 步骤5: 匹配深度文件与Poses

将深度文件与对应的poses和intrinsics匹配，构建c2w和intri列表。


In [11]:
# 根据深度文件直接读取对应的外参和内参
# 重构后的流程：深度文件 -> 解析(frame_idx, cam_id) -> 直接读取对应的外参和内参文件
extrinsics_dir = os.path.join(scene_dir, 'extrinsics')
intrinsics_dir = os.path.join(scene_dir, 'intrinsics')

# 加载参考pose用于对齐（从步骤1）
camera_front_start = pcd_generator.camera_front_start
OPENCV2DATASET = np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]])

# 预先加载所有相机的内参（每个相机固定）
cam_intrinsics_dict = {}
for cam_id in chosen_cam_id:
    intrinsic_file = os.path.join(intrinsics_dir, f'{cam_id}.txt')
    if os.path.exists(intrinsic_file):
        intrinsic_data = np.loadtxt(intrinsic_file)
        fx, fy, cx, cy = intrinsic_data[0], intrinsic_data[1], intrinsic_data[2], intrinsic_data[3]
        # 转换为3x3矩阵
        cam_intrinsic = np.array([[fx, 0, cx], [0, fy, cy], [0, 0, 1]])
        cam_intrinsics_dict[cam_id] = cam_intrinsic
    else:
        CONSOLE.log(f"[yellow]Warning: Intrinsic file not found: {intrinsic_file}[/yellow]")

valid_depth_files = []
pcd_generator.c2w = []
pcd_generator.intri = []

for file_name in depth_files:
    try:
        # 从深度文件名解析frame_idx和cam_id
        parts = file_name.replace('.npy', '').split('_')
        if len(parts) >= 2:
            frame_idx = int(parts[0])
            cam_id = int(parts[1])
            
            # 验证相机ID在chosen_cam_id中
            if cam_id not in chosen_cam_id:
                CONSOLE.log(f"Warning: Camera ID {cam_id} not in chosen_cam_id {chosen_cam_id}, skipping {file_name}")
                continue
            
            # 直接读取对应的外参文件
            extrinsic_file = os.path.join(extrinsics_dir, f'{frame_idx:03d}_{cam_id}.txt')
            if not os.path.exists(extrinsic_file):
                CONSOLE.log(f"Warning: Extrinsic file not found: {extrinsic_file}, skipping {file_name}")
                continue
            
            # 加载外参（cam_to_world）
            cam2world = np.loadtxt(extrinsic_file)
            
            # 对齐到第一帧第一相机（如果可用）
            if camera_front_start is not None:
                cam2world = np.linalg.inv(camera_front_start) @ cam2world
            
            # 转换为OpenCV坐标系
            cam2world = cam2world @ OPENCV2DATASET
            
            # 获取对应的内参
            if cam_id in cam_intrinsics_dict:
                cam_intrinsic = cam_intrinsics_dict[cam_id]
            else:
                CONSOLE.log(f"Warning: Intrinsic not found for cam_id {cam_id}, using identity matrix")
                cam_intrinsic = np.eye(3)
            
            # 添加到列表
            valid_depth_files.append(file_name)
            pcd_generator.c2w.append(cam2world)
            pcd_generator.intri.append(cam_intrinsic)
            
    except (ValueError, IndexError) as e:
        CONSOLE.log(f"Warning: Error processing {file_name}: {e}")
        continue
    except Exception as e:
        CONSOLE.log(f"Warning: Unexpected error processing {file_name}: {e}")
        continue

depth_files = valid_depth_files

CONSOLE.log(f"[cyan]Matching results:[/cyan]")
CONSOLE.log(f"  {sparsity}: ALL frames: {len(depth_files)}")
CONSOLE.log(f"  c2w count: {len(pcd_generator.c2w)}")
CONSOLE.log(f"  intri count: {len(pcd_generator.intri)}")
CONSOLE.log(f"  Chosen camera IDs: {chosen_cam_id}")

if len(depth_files) == 0:
    raise ValueError("No depth files selected after sparsity filtering and camera filtering")

if len(pcd_generator.c2w) != len(depth_files):
    raise ValueError(f"Mismatch: {len(pcd_generator.c2w)} poses for {len(depth_files)} depth files")

if len(pcd_generator.intri) != len(depth_files):
    raise ValueError(f"Mismatch: {len(pcd_generator.intri)} intrinsics for {len(depth_files)} depth files")

CONSOLE.log("[green]Step 5: Matching completed![/green]")
CONSOLE.log("[cyan]Note: Poses and intrinsics are now loaded directly from depth files[/cyan]")


           Matching results:                                                                            ]8;id=865623;file:///tmp/ipykernel_22413/3419919168.py\3419919168.py]8;;\:]8;id=625488;file:///tmp/ipykernel_22413/3419919168.py#77\77]8;;\

             Drop50: ALL frames: 98                                                                     ]8;id=930921;file:///tmp/ipykernel_22413/3419919168.py\3419919168.py]8;;\:]8;id=931236;file:///tmp/ipykernel_22413/3419919168.py#78\78]8;;\

             c2w count: 98                                                                              ]8;id=81179;file:///tmp/ipykernel_22413/3419919168.py\3419919168.py]8;;\:]8;id=865330;file:///tmp/ipykernel_22413/3419919168.py#79\79]8;;\

             intri count: 98                                                                            ]8;id=179267;file:///tmp/ipykernel_22413/3419919168.py\3419919168.py]8;;\:]8;id=587231;file:///tmp/ipykernel_22413/3419919168.py#80\80]8;;\

             Chosen camera IDs: [0]                                                                     ]8;id=87384;file:///tmp/ipykernel_22413/3419919168.py\3419919168.py]8;;\:]8;id=641074;file:///tmp/ipykernel_22413/3419919168.py#81\81]8;;\

           Step 5: Matching completed!                                                                  ]8;id=947280;file:///tmp/ipykernel_22413/3419919168.py\3419919168.py]8;;\:]8;id=736797;file:///tmp/ipykernel_22413/3419919168.py#92\92]8;;\

           Note: Poses and intrinsics are now loaded directly from depth files                          ]8;id=771569;file:///tmp/ipykernel_22413/3419919168.py\3419919168.py]8;;\:]8;id=526924;file:///tmp/ipykernel_22413/3419919168.py#93\93]8;;\

## 步骤6: 深度一致性检查

检查连续帧之间的深度一致性，生成一致性掩码。


In [12]:
# 深度一致性检查
if depth_consistency:
    CONSOLE.log("[cyan]Performing depth consistency check...[/cyan]")
    consistency_mask = pcd_generator.depth_cosistency_check(depth_files=depth_files, H=H, W=W)
    CONSOLE.log(f"[green]Generated {len(consistency_mask)} consistency masks[/green]")
else:
    consistency_mask = [np.ones((H, W)) for _ in range(len(depth_files))]
    CONSOLE.log("[yellow]Depth consistency check disabled[/yellow]")

# 显示一致性掩码统计
if len(consistency_mask) > 0:
    avg_valid_ratio = np.mean([mask.sum() / mask.size for mask in consistency_mask])
    CONSOLE.log(f"[cyan]Consistency mask statistics:[/cyan]")
    CONSOLE.log(f"  Average valid ratio: {avg_valid_ratio:.2%}")

CONSOLE.log("[green]Step 6: Depth consistency check completed![/green]")


           Performing depth consistency check...                                                         ]8;id=885348;file:///tmp/ipykernel_22413/1612303207.py\1612303207.py]8;;\:]8;id=357152;file:///tmp/ipykernel_22413/1612303207.py#3\3]8;;\

Depth Consistency Check!


[16:29:08] Generated 98 consistency masks                                                                ]8;id=483575;file:///tmp/ipykernel_22413/1612303207.py\1612303207.py]8;;\:]8;id=965473;file:///tmp/ipykernel_22413/1612303207.py#5\5]8;;\

[16:29:09] Consistency mask statistics:                                                                 ]8;id=308234;file:///tmp/ipykernel_22413/1612303207.py\1612303207.py]8;;\:]8;id=951580;file:///tmp/ipykernel_22413/1612303207.py#13\13]8;;\

             Average valid ratio: 86.43%                                                                ]8;id=328158;file:///tmp/ipykernel_22413/1612303207.py\1612303207.py]8;;\:]8;id=184222;file:///tmp/ipykernel_22413/1612303207.py#14\14]8;;\

           Step 6: Depth consistency check completed!                                                   ]8;id=664877;file:///tmp/ipykernel_22413/1612303207.py\1612303207.py]8;;\:]8;id=707021;file:///tmp/ipykernel_22413/1612303207.py#16\16]8;;\

## 步骤7: 点云累积主循环

这是核心步骤，将深度图转换为点云。包括：
- 加载和预处理深度图（使用depth_utils）
- 加载RGB图像
- 应用各种掩码（天空、一致性、下采样）
- 反投影到相机坐标系
- 变换到世界坐标系
- 累积点云


In [18]:
# 初始化下采样掩码
color_pointclouds = []
if downscale != 1:
    downscale_mask = np.zeros_like(consistency_mask[0])
    downscale_mask[::downscale, ::downscale] = 1
    CONSOLE.log(f"[cyan]Downscale mask created: sampling every {downscale} pixels[/cyan]")
else:
    downscale_mask = None

CONSOLE.log(f"[cyan]Starting point cloud accumulation for {len(depth_files)} depth files...[/cyan]")

# 遍历所有深度文件
for i, file_name in enumerate(depth_files):
    if (i + 1) % 10 == 0 or i == 0:
        CONSOLE.log(f"Processing {i+1}/{len(depth_files)}: {file_name}")
    
    # 7.1: 加载和预处理深度图（使用depth_utils）
    depth_file = os.path.join(pcd_generator.depth_dir, file_name)
    try:
        # 重要：使用depth_utils进行预处理，自动插值到原始尺寸
        depth, metadata = process_depth_for_use(depth_file, target_shape=(H, W))
    except Exception as e:
        CONSOLE.log(f"Warning: Failed to load depth {file_name}: {e}")
        continue
    
    # 7.2: 加载RGB图像
    rgb_file = os.path.join(scene_dir, 'images', file_name.replace('.npy', '.jpg'))
    if not os.path.exists(rgb_file):
        rgb_file = os.path.join(scene_dir, 'images', file_name.replace('.npy', '.png'))
    
    if not os.path.exists(rgb_file):
        CONSOLE.log(f"Warning: RGB file not found for {file_name}")
        continue
    
    try:
        rgb = imageio.imread(rgb_file) / 255.0
    except Exception as e:
        CONSOLE.log(f"Warning: Failed to load RGB {file_name}: {e}")
        continue
    
    # 7.3: 应用天空过滤（如果启用）
    if filter_sky:
        sky_mask_file = os.path.join(
            scene_dir, 'sky_masks', 
            file_name.replace('.npy', '.png')
        )
        if os.path.exists(sky_mask_file):
            sky_mask = cv2.imread(sky_mask_file, cv2.IMREAD_GRAYSCALE)
            # sky_masks: 0 = sky, 255 = non-sky
            # Convert to boolean: True = non-sky (keep), False = sky (filter)
            mask = (sky_mask > 0).astype(np.bool_)
            final_mask = np.logical_and(consistency_mask[i], mask)
        else:
            CONSOLE.log(f"Warning: Sky mask not found: {sky_mask_file}, skipping sky filtering")
            final_mask = consistency_mask[i]
    else:
        final_mask = consistency_mask[i]
    
    # 7.4: 应用下采样掩码
    if downscale != 1:
        final_mask = np.logical_and(downscale_mask, final_mask)
    
    # 7.5: 提取有效像素
    kept = np.argwhere(final_mask)
    
    if len(kept) == 0:
        continue
    
    depth_values = depth[kept[:, 0], kept[:, 1]]
    rgb_values = rgb[kept[:, 0], kept[:, 1]]
    
    # 7.6: 过滤无效深度值
    valid_depth_mask = np.isfinite(depth_values) & (depth_values > 0)
    if not np.any(valid_depth_mask):
        continue
    
    depth_values = depth_values[valid_depth_mask]
    rgb_values = rgb_values[valid_depth_mask]
    kept_valid = kept[valid_depth_mask]
    
    try:
        c2w = pcd_generator.c2w[i]
        K = pcd_generator.intri[i]
    except IndexError as e:
        CONSOLE.log(f"Warning: Index error for {file_name}: {e}")
        continue
    
    # 7.7: 反投影到相机坐标系
    # 生成像素坐标网格
    x = np.arange(0, W)
    y = np.arange(0, H)
    xx, yy = np.meshgrid(x, y)
    pixels = np.vstack((xx.ravel(), yy.ravel())).T.reshape(H, W, 2)
    
    # 反投影深度到3D点
    # 确保K是3x3矩阵（兼容旧代码中可能的4x4格式）
    if K.shape == (4, 4):
        K_3x3 = K[:3, :3]
    elif K.shape == (3, 3):
        K_3x3 = K
    else:
        # Fallback: 尝试提取3x3部分
        K_3x3 = K[:3, :3] if K.shape[0] >= 3 and K.shape[1] >= 3 else K
        CONSOLE.log(f"Warning: Unexpected intrinsics shape {K.shape} for {file_name}, using {K_3x3.shape}")
    
    pixel_coords = pixels[kept_valid[:, 0], kept_valid[:, 1]]
    x_cam = (pixel_coords[:, 0] - K_3x3[0, 2]) * depth_values / K_3x3[0, 0]
    y_cam = (pixel_coords[:, 1] - K_3x3[1, 2]) * depth_values / K_3x3[1, 1]
    z_cam = depth_values
    coordinates = np.stack([x_cam, y_cam, z_cam], axis=1)
    
    # 过滤NaN/inf坐标
    valid_coords_mask = np.isfinite(coordinates).all(axis=1)
    if not np.any(valid_coords_mask):
        continue
    
    coordinates = coordinates[valid_coords_mask]
    rgb_values = rgb_values[valid_coords_mask]
    coordinates = np.column_stack((coordinates, np.ones(len(coordinates))))
    
    # 7.8: 变换到世界坐标系
    worlds = np.dot(c2w, coordinates.T).T
    worlds = worlds[:, :3]
    
    # 过滤NaN/inf世界坐标
    valid_worlds_mask = np.isfinite(worlds).all(axis=1)
    if not np.any(valid_worlds_mask):
        continue
    
    worlds = worlds[valid_worlds_mask]
    rgb_values = rgb_values[valid_worlds_mask]
    
    # 7.9: 累积点云块
    point_cloud_chunk = np.concatenate([worlds, rgb_values.reshape(-1, 3)], axis=-1)
    color_pointclouds.append(point_cloud_chunk)

# 7.10: 合并所有点云块
if len(color_pointclouds) == 0:
    raise ValueError("No valid point cloud generated")

accumulated_pointcloud = np.concatenate(color_pointclouds, axis=0).reshape(-1, 6)

# 最终过滤：移除剩余的NaN/inf值
valid_mask = np.isfinite(accumulated_pointcloud[:, :3]).all(axis=1)
if not np.any(valid_mask):
    raise ValueError("No valid point cloud after NaN filtering")

accumulated_pointcloud = accumulated_pointcloud[valid_mask]

CONSOLE.log(f"[green]Step 7: Point cloud accumulation completed![/green]")
CONSOLE.log(f"  Total points: {len(accumulated_pointcloud)}")
CONSOLE.log(f"  Point cloud shape: {accumulated_pointcloud.shape}")

# 保存步骤7的中间点云（累积后）
ply_dir = os.path.join(scene_dir, 'ply')
tmp_dir = os.path.join(ply_dir, 'tmp')
os.makedirs(tmp_dir, exist_ok=True)

step7_pcd = o3d.geometry.PointCloud()
step7_pcd.points = o3d.utility.Vector3dVector(accumulated_pointcloud[:, :3])
step7_pcd.colors = o3d.utility.Vector3dVector(accumulated_pointcloud[:, 3:])
step7_file = os.path.join(tmp_dir, f'step7_accumulated_{frame_start}.ply')
o3d.io.write_point_cloud(step7_file, step7_pcd)
CONSOLE.log(f"[cyan]Saved step 7 point cloud: {step7_file}[/cyan]")

# 显示步骤7的点云
# 启用自动重载模块（修改tools.plyviewer后自动重新加载）
%load_ext autoreload
%autoreload 2

try:
    from tools.plyviewer import PLYViewer
    # 创建或获取全局viewer实例（所有步骤的点云都会添加到同一个viewer中）
    # 可以通过scene tree调整各个点云的可见性
    global_viewer = PLYViewer.get_or_create_global_viewer(
        host="0.0.0.0", 
        port=7007, 
        point_size=0.01, 
        auto_fallback=True
    )
    global_viewer.start_viewer()
    global_viewer.load_and_display(step7_file, name="/step7_accumulated", visible=True)
    CONSOLE.log("[green]✓ Step 7 point cloud added to viewer[/green]")
    CONSOLE.log(f"[cyan]Viewer URL: {global_viewer.viewer_info}[/cyan]")
    CONSOLE.log("[yellow]Note: All point clouds will be added to the same viewer. Use scene tree to toggle visibility.[/yellow]")
except Exception as e:
    CONSOLE.log(f"[yellow]Warning: Could not add step 7 point cloud to viewer: {e}[/yellow]")


[16:38:16] Downscale mask created: sampling every 2 pixels                                               ]8;id=683681;file:///tmp/ipykernel_22413/3754134746.py\3754134746.py]8;;\:]8;id=275556;file:///tmp/ipykernel_22413/3754134746.py#6\6]8;;\

           Starting point cloud accumulation for 98 depth files...                                      ]8;id=757854;file:///tmp/ipykernel_22413/3754134746.py\3754134746.py]8;;\:]8;id=582213;file:///tmp/ipykernel_22413/3754134746.py#10\10]8;;\

           Processing 1/98: 000_0.npy                                                                   ]8;id=222420;file:///tmp/ipykernel_22413/3754134746.py\3754134746.py]8;;\:]8;id=723124;file:///tmp/ipykernel_22413/3754134746.py#15\15]8;;\

[16:38:17] Processing 10/98: 017_0.npy                                                                  ]8;id=578503;file:///tmp/ipykernel_22413/3754134746.py\3754134746.py]8;;\:]8;id=919708;file:///tmp/ipykernel_22413/3754134746.py#15\15]8;;\

[16:38:18] Processing 20/98: 037_0.npy                                                                  ]8;id=501970;file:///tmp/ipykernel_22413/3754134746.py\3754134746.py]8;;\:]8;id=40503;file:///tmp/ipykernel_22413/3754134746.py#15\15]8;;\

...

[16:39:04] ✓ Step 7 point cloud added to viewer                                                        ]8;id=154022;file:///tmp/ipykernel_22413/3754134746.py\3754134746.py]8;;\:]8;id=487744;file:///tmp/ipykernel_22413/3754134746.py#183\183]8;;\

           Viewer URL: Viewer running locally at: http://localhost:7007 (listening on 0.0.0.0)         ]8;id=689267;file:///tmp/ipykernel_22413/3754134746.py\3754134746.py]8;;\:]8;id=575906;file:///tmp/ipykernel_22413/3754134746.py#184\184]8;;\

           Note: All point clouds will be added to the same viewer. Use scene tree to toggle           ]8;id=248766;file:///tmp/ipykernel_22413/3754134746.py\3754134746.py]8;;\:]8;id=112076;file:///tmp/ipykernel_22413/3754134746.py#185\185]8;;\
           visibility.                                                                                                  

(viser) Connection closed (0, 0 total)

(viser) Connection opened (0, 1 total), 7 persistent messages

In [19]:
points = accumulated_pointcloud[:, :3]
colors = accumulated_pointcloud[:, 3:]

if use_bbx:
    bbx_min, bbx_max = pcd_generator.get_bbx()
    CONSOLE.log(f"[cyan]Bounding box range:[/cyan]")
    CONSOLE.log(f"  Min: {bbx_min}")
    CONSOLE.log(f"  Max: {bbx_max}")
    
    # 裁剪点云
    points, colors = pcd_generator.crop_pointcloud(bbx_min, bbx_max, points, colors)
    
    # 分离内部和外部点云
    inside_pnt, inside_rgb, outside_pnt, outside_rgb = pcd_generator.split_pointcloud(
        bbx_min, bbx_max, points, colors
    )
    
    CONSOLE.log(f"[cyan]Bounding box filtering results:[/cyan]")
    CONSOLE.log(f"  Inside points: {len(inside_pnt)}")
    CONSOLE.log(f"  Outside points: {len(outside_pnt)}")
    CONSOLE.log(f"  Total after crop: {len(points)}")
else:
    CONSOLE.log("[yellow]Bounding box filtering disabled[/yellow]")
    inside_pnt, inside_rgb, outside_pnt, outside_rgb = None, None, None, None

CONSOLE.log("[green]Step 8: Bounding box cropping completed![/green]")

# 保存步骤8的中间点云（裁剪后）
# 确保tmp_dir已定义（在步骤7中已创建）
ply_dir = os.path.join(scene_dir, 'ply')
tmp_dir = os.path.join(ply_dir, 'tmp')
os.makedirs(tmp_dir, exist_ok=True)

if len(points) > 0:
    step8_pcd = o3d.geometry.PointCloud()
    step8_pcd.points = o3d.utility.Vector3dVector(points[:, :3])
    step8_pcd.colors = o3d.utility.Vector3dVector(colors)
    step8_file = os.path.join(tmp_dir, f'step8_cropped_{frame_start}.ply')
    o3d.io.write_point_cloud(step8_file, step8_pcd)
    CONSOLE.log(f"[cyan]Saved step 8 point cloud: {step8_file}[/cyan]")
    
    # 显示步骤8的点云（添加到全局viewer）
    try:
        from tools.plyviewer import PLYViewer
        # 使用全局viewer实例，添加step8的点云
        global_viewer = PLYViewer.get_or_create_global_viewer()
        global_viewer.load_and_display(step8_file, name="/step8_cropped", visible=False)
        CONSOLE.log("[green]✓ Step 8 point cloud added to viewer[/green]")
        CONSOLE.log("[yellow]Note: Step 8 point cloud is hidden by default. Use scene tree to toggle visibility.[/yellow]")
    except Exception as e:
        CONSOLE.log(f"[yellow]Warning: Could not add step 8 point cloud to viewer: {e}[/yellow]")


[16:40:44] Bounding box range:                                                                           ]8;id=461189;file:///tmp/ipykernel_22413/1968458330.py\1968458330.py]8;;\:]8;id=689208;file:///tmp/ipykernel_22413/1968458330.py#6\6]8;;\

             Min: [-20 -20 -20]                                                                          ]8;id=985674;file:///tmp/ipykernel_22413/1968458330.py\1968458330.py]8;;\:]8;id=817228;file:///tmp/ipykernel_22413/1968458330.py#7\7]8;;\

             Max: [20.   4.8 70. ]                                                                       ]8;id=71256;file:///tmp/ipykernel_22413/1968458330.py\1968458330.py]8;;\:]8;id=171580;file:///tmp/ipykernel_22413/1968458330.py#8\8]8;;\

[16:40:45] Bounding box filtering results:                                                              ]8;id=81129;file:///tmp/ipykernel_22413/1968458330.py\1968458330.py]8;;\:]8;id=867855;file:///tmp/ipykernel_22413/1968458330.py#18\18]8;;\

             Inside points: 4646383                                                                     ]8;id=778507;file:///tmp/ipykernel_22413/1968458330.py\1968458330.py]8;;\:]8;id=706239;file:///tmp/ipykernel_22413/1968458330.py#19\19]8;;\

...

             Total after crop: 4646405                                                                  ]8;id=128432;file:///tmp/ipykernel_22413/1968458330.py\1968458330.py]8;;\:]8;id=64000;file:///tmp/ipykernel_22413/1968458330.py#21\21]8;;\

           Step 8: Bounding box cropping completed!                                                     ]8;id=825486;file:///tmp/ipykernel_22413/1968458330.py\1968458330.py]8;;\:]8;id=294057;file:///tmp/ipykernel_22413/1968458330.py#26\26]8;;\

[16:40:46] Saved step 8 point cloud:                                                                    ]8;id=834022;file:///tmp/ipykernel_22413/1968458330.py\1968458330.py]8;;\:]8;id=849971;file:///tmp/ipykernel_22413/1968458330.py#40\40]8;;\
           /root/autodl-tmp/nuScenes/preprocess/trainval/000/ply/tmp/step8_cropped_0.ply                                

[16:40:46] Created global PLYViewer instance                                                                                                                                                            ]8;id=959785;file:///root/drivestudio-coding/tools/plyviewer.py\plyviewer.py]8;;\:]8;id=162263;file:///root/drivestudio-coding/tools/plyviewer.py#100\100]8;;\
           Loading PLY file: /root/autodl-tmp/nuScenes/preprocess/trainval/000/ply/tmp/step8_cropped_0.ply                                                                                              ]8;id=787955;file:///root/drivestudio-coding/tools/plyviewer.py\plyviewer.py]8;;\:]8;id=376129;file:///root/drivestudio-coding/tools/plyviewer.py#336\336]8;;\
[16:40:47] ✓ Loaded 4646405 points                                                                                                                                                                      ]8;id=758096;file:///root/drivestudio-coding/tools/plyviewer.py\

[16:40:47] Warning: Could not add step 8 point cloud to viewer: Viewer not started. Call start_viewer() ]8;id=673615;file:///tmp/ipykernel_22413/1968458330.py\1968458330.py]8;;\:]8;id=310655;file:///tmp/ipykernel_22413/1968458330.py#51\51]8;;\
           first.                                                                                                       

## 步骤9: 点云滤波

对点云进行统计滤波和均匀下采样，去除噪声。


In [20]:
if use_bbx:
    # 内部点云滤波
    inside_pointcloud = o3d.geometry.PointCloud()
    inside_pointcloud.points = o3d.utility.Vector3dVector(inside_pnt[:, :3])
    inside_pointcloud.colors = o3d.utility.Vector3dVector(inside_rgb)
    
    CONSOLE.log("[cyan]Filtering inside point cloud...[/cyan]")
    cl, ind = inside_pointcloud.remove_statistical_outlier(
        nb_neighbors=35, std_ratio=1.5
    )
    inside_pointcloud = inside_pointcloud.select_by_index(ind)
    CONSOLE.log(f"  Inside points after filtering: {len(inside_pointcloud.points)}")
    
    # 外部点云滤波
    outside_pointcloud = o3d.geometry.PointCloud()
    outside_pointcloud.points = o3d.utility.Vector3dVector(outside_pnt[:, :3])
    outside_pointcloud.colors = o3d.utility.Vector3dVector(outside_rgb)
    
    CONSOLE.log("[cyan]Filtering outside point cloud...[/cyan]")
    cl, ind = outside_pointcloud.remove_statistical_outlier(
        nb_neighbors=20, std_ratio=2.0
    )
    outside_pointcloud = outside_pointcloud.select_by_index(ind)
    CONSOLE.log(f"  Outside points after filtering: {len(outside_pointcloud.points)}")
    
    # 合并并下采样
    combined_pointcloud = inside_pointcloud + outside_pointcloud
    combined_pointcloud = combined_pointcloud.uniform_down_sample(every_k_points=2)
    final_pointcloud = combined_pointcloud
    
    CONSOLE.log(f"  Final points after downsampling: {len(final_pointcloud.points)}")
else:
    # 全局滤波
    point_cloud = o3d.geometry.PointCloud()
    point_cloud.points = o3d.utility.Vector3dVector(points[:, :3])
    point_cloud.colors = o3d.utility.Vector3dVector(colors)
    
    CONSOLE.log("[cyan]Filtering point cloud...[/cyan]")
    cl, ind = point_cloud.remove_statistical_outlier(
        nb_neighbors=30, std_ratio=1.5
    )
    point_cloud = point_cloud.select_by_index(ind)
    point_cloud = point_cloud.uniform_down_sample(every_k_points=3)
    final_pointcloud = point_cloud
    
    CONSOLE.log(f"  Final points: {len(final_pointcloud.points)}")

CONSOLE.log("[green]Step 9: Point cloud filtering completed![/green]")

# 保存步骤9的中间点云（滤波后）
step9_file = os.path.join(tmp_dir, f'step9_filtered_{frame_start}.ply')
o3d.io.write_point_cloud(step9_file, final_pointcloud)
CONSOLE.log(f"[cyan]Saved step 9 point cloud: {step9_file}[/cyan]")

# 显示步骤9的点云（添加到全局viewer）
try:
    from tools.plyviewer import PLYViewer
    # 使用全局viewer实例，添加step9的点云
    global_viewer = PLYViewer.get_or_create_global_viewer()
    global_viewer.load_and_display(step9_file, name="/step9_filtered", visible=False)
    CONSOLE.log("[green]✓ Step 9 point cloud added to viewer[/green]")
    CONSOLE.log("[yellow]Note: Step 9 point cloud is hidden by default. Use scene tree to toggle visibility.[/yellow]")
except Exception as e:
    CONSOLE.log(f"[yellow]Warning: Could not add step 9 point cloud to viewer: {e}[/yellow]")


           Filtering inside point cloud...                                                               ]8;id=264374;file:///tmp/ipykernel_22413/1413671000.py\1413671000.py]8;;\:]8;id=92225;file:///tmp/ipykernel_22413/1413671000.py#7\7]8;;\

[16:40:51]   Inside points after filtering: 4467846                                                     ]8;id=999892;file:///tmp/ipykernel_22413/1413671000.py\1413671000.py]8;;\:]8;id=761487;file:///tmp/ipykernel_22413/1413671000.py#12\12]8;;\

           Filtering outside point cloud...                                                             ]8;id=233262;file:///tmp/ipykernel_22413/1413671000.py\1413671000.py]8;;\:]8;id=241980;file:///tmp/ipykernel_22413/1413671000.py#19\19]8;;\

             Outside points after filtering: 22                                                         ]8;id=358422;file:///tmp/ipykernel_22413/1413671000.py\1413671000.py]8;;\:]8;id=907730;file:///tmp/ipykernel_22413/1413671000.py#24\24]8;;\

             Final points after downsampling: 2233934                                                   ]8;id=959786;file:///tmp/ipykernel_22413/1413671000.py\1413671000.py]8;;\:]8;id=460944;file:///tmp/ipykernel_22413/1413671000.py#31\31]8;;\

...

(viser) Connection opened (0, 1 total), 7 persistent messages

(viser) Connection closed (0, 0 total)

(viser) Server stopped

(viser) Server stopped

(viser) Connection closed (0, 0 total)

## 步骤10: 保存点云

将处理后的点云保存为PLY格式。


In [16]:
# 创建保存目录（最终点云保存在ply文件夹中）
ply_dir = os.path.join(scene_dir, 'ply')
os.makedirs(ply_dir, exist_ok=True)

# 保存最终点云
output_file = os.path.join(ply_dir, f'{frame_start}.ply')
try:
    o3d.io.write_point_cloud(output_file, final_pointcloud)
    CONSOLE.log(f"[green]Point cloud saved successfully![/green]")
    CONSOLE.log(f"  Output file: {output_file}")
    CONSOLE.log(f"  Point count: {len(final_pointcloud.points)}")
except Exception as e:
    CONSOLE.log(f"[red]Error saving point cloud: {e}[/red]")
    raise

CONSOLE.log("[green]Step 10: Point cloud saving completed![/green]")

# 显示最终点云（添加到全局viewer）
try:
    from tools.plyviewer import PLYViewer
    # 使用全局viewer实例，添加最终点云
    global_viewer = PLYViewer.get_or_create_global_viewer()
    global_viewer.load_and_display(output_file, name="/final_pointcloud", visible=False)
    CONSOLE.log("[green]✓ Final point cloud added to viewer[/green]")
    CONSOLE.log(f"[cyan]Viewer URL: {global_viewer.viewer_info}[/cyan]")
    CONSOLE.log(f"[cyan]Total point clouds in viewer: {len(global_viewer.get_loaded_point_clouds())}[/cyan]")
    CONSOLE.log(f"[cyan]Loaded point clouds: {', '.join(global_viewer.get_loaded_point_clouds())}[/cyan]")
    CONSOLE.log("[yellow]Note: Final point cloud is hidden by default. Use scene tree to toggle visibility.[/yellow]")
    CONSOLE.log("[yellow]Tip: You can show/hide different steps by clicking the eye icon in the scene tree.[/yellow]")
except Exception as e:
    CONSOLE.log(f"[yellow]Warning: Could not add final point cloud to viewer: {e}[/yellow]")


           Point cloud saved successfully!                                                               ]8;id=618483;file:///tmp/ipykernel_22413/3775801549.py\3775801549.py]8;;\:]8;id=566176;file:///tmp/ipykernel_22413/3775801549.py#9\9]8;;\

             Output file: /root/autodl-tmp/nuScenes/preprocess/trainval/000/ply/0.ply                   ]8;id=315002;file:///tmp/ipykernel_22413/3775801549.py\3775801549.py]8;;\:]8;id=951804;file:///tmp/ipykernel_22413/3775801549.py#10\10]8;;\

             Point count: 2233934                                                                       ]8;id=43869;file:///tmp/ipykernel_22413/3775801549.py\3775801549.py]8;;\:]8;id=258395;file:///tmp/ipykernel_22413/3775801549.py#11\11]8;;\

           Step 10: Point cloud saving completed!                                                       ]8;id=519917;file:///tmp/ipykernel_22413/3775801549.py\3775801549.py]8;;\:]8;id=908832;file:///tmp/ipykernel_22413/3775801549.py#16\16]8;;\

           Using existing global PLYViewer instance                                                ]8;id=966767;file:///root/drivestudio-coding/tools/plyviewer.py\plyviewer.py]8;;\:]8;id=634619;file:///root/drivestudio-coding/tools/plyviewer.py#105\105]8;;\

...

           Viewer URL: Viewer running locally at: http://localhost:53513 (listening on 0.0.0.0)         ]8;id=968329;file:///tmp/ipykernel_22413/3775801549.py\3775801549.py]8;;\:]8;id=968778;file:///tmp/ipykernel_22413/3775801549.py#25\25]8;;\

           Total point clouds in viewer: 4                                                              ]8;id=634647;file:///tmp/ipykernel_22413/3775801549.py\3775801549.py]8;;\:]8;id=78629;file:///tmp/ipykernel_22413/3775801549.py#26\26]8;;\

           Loaded point clouds: /step7_accumulated, /step8_cropped, /step9_filtered, /final_pointcloud  ]8;id=542330;file:///tmp/ipykernel_22413/3775801549.py\3775801549.py]8;;\:]8;id=225694;file:///tmp/ipykernel_22413/3775801549.py#27\27]8;;\

           Note: Final point cloud is hidden by default. Use scene tree to toggle visibility.           ]8;id=830211;file:///tmp/ipykernel_22413/3775801549.py\3775801549.py]8;;\:]8;id=965782;file:///tmp/ipykernel_22413/3775801549.py#28\28]8;;\

           Tip: You can show/hide different steps by clicking the eye icon in the scene tree.           ]8;id=153534;file:///tmp/ipykernel_22413/3775801549.py\3775801549.py]8;;\:]8;id=975825;file:///tmp/ipykernel_22413/3775801549.py#29\29]8;;\

## 反直觉检查

检查可能的问题和潜在错误。


In [ ]:
CONSOLE.log("[cyan]Performing counterintuitive checks...[/cyan]")

# 1. 检查深度图尺寸
CONSOLE.log("\n[cyan]1. Depth map size check:[/cyan]")
sample_depth_file = os.path.join(pcd_generator.depth_dir, depth_files[0])
try:
    depth_raw, metadata = load_depth_with_metadata(sample_depth_file)
    depth_processed, _ = process_depth_for_use(sample_depth_file, target_shape=(H, W))
    CONSOLE.log(f"  Raw depth shape: {depth_raw.shape}")
    CONSOLE.log(f"  Processed depth shape: {depth_processed.shape}")
    CONSOLE.log(f"  Target shape: ({H}, {W})")
    CONSOLE.log(f"  Metadata ori_shape: {metadata.get('ori_shape', 'N/A')}")
    if depth_raw.shape != (H, W):
        CONSOLE.log(f"  [yellow]⚠ Depth map needs interpolation (raw != target)[/yellow]")
    else:
        CONSOLE.log(f"  [green]✓ Depth map size matches target[/green]")
except Exception as e:
    CONSOLE.log(f"  [red]✗ Error checking depth size: {e}[/red]")

# 2. 检查文件名解析和相机ID
CONSOLE.log("\n[cyan]2. Filename parsing and camera ID check:[/cyan]")
sample_files = depth_files[:5]
found_cam_ids = set()
for f in sample_files:
    try:
        parts = f.replace('.npy', '').split('_')
        if len(parts) >= 2:
            frame_idx = int(parts[0])
            cam_id = int(parts[1])
            found_cam_ids.add(cam_id)
            in_chosen = "✓" if cam_id in chosen_cam_id else "✗"
            CONSOLE.log(f"  {f} -> frame_idx={frame_idx}, cam_id={cam_id} {in_chosen}")
        else:
            CONSOLE.log(f"  [red]✗ Cannot parse: {f}[/red]")
    except Exception as e:
        CONSOLE.log(f"  [red]✗ Error parsing {f}: {e}[/red]")

# 检查所有深度文件的相机ID是否都在chosen_cam_id中
all_depth_cam_ids = set()
for f in depth_files:
    try:
        parts = f.replace('.npy', '').split('_')
        if len(parts) >= 2:
            all_depth_cam_ids.add(int(parts[1]))
    except:
        continue

if all_depth_cam_ids.issubset(set(chosen_cam_id)):
    CONSOLE.log(f"  [green]✓ All depth file camera IDs ({all_depth_cam_ids}) are in chosen_cam_id ({chosen_cam_id})[/green]")
else:
    unexpected = all_depth_cam_ids - set(chosen_cam_id)
    CONSOLE.log(f"  [red]✗ Found unexpected camera IDs in depth files: {unexpected}[/red]")
    CONSOLE.log(f"  [yellow]⚠ This may cause matching issues. Check chosen_cam_id configuration.[/yellow]")

# 3. 检查数量匹配
CONSOLE.log("\n[cyan]3. Count matching check:[/cyan]")
CONSOLE.log(f"  Poses count: {len(poses)}")
CONSOLE.log(f"  Intrinsics count: {len(intrinsics)}")
CONSOLE.log(f"  Depth files count: {len(depth_files)}")
CONSOLE.log(f"  c2w count: {len(pcd_generator.c2w)}")
CONSOLE.log(f"  intri count: {len(pcd_generator.intri)}")
if len(pcd_generator.c2w) == len(depth_files) == len(pcd_generator.intri):
    CONSOLE.log(f"  [green]✓ All counts match[/green]")
else:
    CONSOLE.log(f"  [red]✗ Count mismatch detected![/red]")

# 4. 检查深度值范围
CONSOLE.log("\n[cyan]4. Depth value range check:[/cyan]")
try:
    sample_depth, _ = process_depth_for_use(sample_depth_file, target_shape=(H, W))
    valid_depths = sample_depth[sample_depth > 0]
    if len(valid_depths) > 0:
        CONSOLE.log(f"  Valid depth range: [{valid_depths.min():.2f}, {valid_depths.max():.2f}] meters")
        CONSOLE.log(f"  Mean depth: {valid_depths.mean():.2f} meters")
        if valid_depths.max() > 200:
            CONSOLE.log(f"  [yellow]⚠ Some depth values exceed 200m (may be outliers)[/yellow]")
        else:
            CONSOLE.log(f"  [green]✓ Depth values in reasonable range[/green]")
    else:
        CONSOLE.log(f"  [red]✗ No valid depth values found[/red]")
except Exception as e:
    CONSOLE.log(f"  [red]✗ Error checking depth range: {e}[/red]")

# 5. 检查坐标变换
CONSOLE.log("\n[cyan]5. Coordinate transformation check:[/cyan]")
sample_c2w = pcd_generator.c2w[0]
CONSOLE.log(f"  Sample c2w matrix shape: {sample_c2w.shape}")
CONSOLE.log(f"  Determinant: {np.linalg.det(sample_c2w[:3, :3]):.6f}")
if abs(np.linalg.det(sample_c2w[:3, :3]) - 1.0) < 0.01:
    CONSOLE.log(f"  [green]✓ Rotation matrix is valid (det ≈ 1)[/green]")
else:
    CONSOLE.log(f"  [yellow]⚠ Rotation matrix determinant is not 1[/yellow]")

# 6. 检查掩码应用顺序
CONSOLE.log("\n[cyan]6. Mask application order check:[/cyan]")
CONSOLE.log(f"  Order: consistency -> sky -> downscale")
CONSOLE.log(f"  [green]✓ Correct order maintained[/green]")

# 7. 检查深度图元数据
CONSOLE.log("\n[cyan]7. Depth metadata check:[/cyan]")
meta_file = sample_depth_file.replace('.npy', '_meta.npz')
if os.path.exists(meta_file):
    meta = dict(np.load(meta_file))
    CONSOLE.log(f"  [green]✓ Metadata file exists[/green]")
    CONSOLE.log(f"  Keys: {list(meta.keys())}")
else:
    CONSOLE.log(f"  [yellow]⚠ Metadata file not found: {meta_file}[/yellow]")

# 8. 检查内参格式
CONSOLE.log("\n[cyan]8. Intrinsic format check:[/cyan]")
sample_K = pcd_generator.intri[0]
CONSOLE.log(f"  Intrinsic matrix shape: {sample_K.shape}")
CONSOLE.log(f"  fx={sample_K[0,0]:.2f}, fy={sample_K[1,1]:.2f}")
CONSOLE.log(f"  cx={sample_K[0,2]:.2f}, cy={sample_K[1,2]:.2f}")
if sample_K.shape == (4, 4):
    CONSOLE.log(f"  [green]✓ Intrinsic is 4x4 matrix format[/green]")
else:
    CONSOLE.log(f"  [yellow]⚠ Intrinsic is not 4x4 format[/yellow]")

CONSOLE.log("\n[green]Counterintuitive checks completed![/green]")


           Performing counterintuitive checks...                                                         ]8;id=651307;file:///tmp/ipykernel_22413/3532195661.py\3532195661.py]8;;\:]8;id=983420;file:///tmp/ipykernel_22413/3532195661.py#1\1]8;;\

                                                                                                         ]8;id=579103;file:///tmp/ipykernel_22413/3532195661.py\3532195661.py]8;;\:]8;id=202306;file:///tmp/ipykernel_22413/3532195661.py#4\4]8;;\
           1. Depth map size check:                                                                                     

             Raw depth shape: (598, 1064)                                                                ]8;id=505799;file:///tmp/ipykernel_22413/3532195661.py\3532195661.py]8;;\:]8;id=187613;file:///tmp/ipykernel_22413/3532195661.py#9\9]8;;\

             Processed depth shape: (900, 1600)                                                         ]8;id=493279;file:///tmp/ipykernel_22413/3532195661.py\3532195661.py]8;;\:]8;id=347118;file:///tmp/ipykernel_22413/3532195661.py#10\10]8;;\

             Target shape: (900, 1600)                                                                  ]8;id=527959;file:///tmp/ipykernel_22413/3532195661.py\3532195661.py]8;;\:]8;id=717651;file:///tmp/ipykernel_22413/3532195661.py#11\11]8;;\

...

             008_0.npy -> frame_idx=8, cam_id=0 ✓                                                       ]8;id=197644;file:///tmp/ipykernel_22413/3532195661.py\3532195661.py]8;;\:]8;id=792803;file:///tmp/ipykernel_22413/3532195661.py#32\32]8;;\

             ✓ All depth file camera IDs ({0}) are in chosen_cam_id ([0])                               ]8;id=253995;file:///tmp/ipykernel_22413/3532195661.py\3532195661.py]8;;\:]8;id=999565;file:///tmp/ipykernel_22413/3532195661.py#49\49]8;;\

                                                                                                        ]8;id=753538;file:///tmp/ipykernel_22413/3532195661.py\3532195661.py]8;;\:]8;id=950331;file:///tmp/ipykernel_22413/3532195661.py#56\56]8;;\
           3. Count matching check:                                                                                     

NameError: name 'poses' is not defined

(viser) Connection opened (0, 1 total), 13 persistent messages

In [ ]:
# 验证输出文件
if os.path.exists(output_file):
    CONSOLE.log(f"[green]✓ Output file exists: {output_file}[/green]")
    
    # 读取点云以验证
    loaded_pcd = o3d.io.read_point_cloud(output_file)
    
    CONSOLE.log(f"[cyan]Point cloud statistics:[/cyan]")
    CONSOLE.log(f"  Point count: {len(loaded_pcd.points)}")
    
    if len(loaded_pcd.points) > 0:
        points_array = np.asarray(loaded_pcd.points)
        CONSOLE.log(f"  X range: [{points_array[:, 0].min():.2f}, {points_array[:, 0].max():.2f}]")
        CONSOLE.log(f"  Y range: [{points_array[:, 1].min():.2f}, {points_array[:, 1].max():.2f}]")
        CONSOLE.log(f"  Z range: [{points_array[:, 2].min():.2f}, {points_array[:, 2].max():.2f}]")
        
        if loaded_pcd.has_colors():
            CONSOLE.log(f"  [green]✓ Point cloud has colors[/green]")
        else:
            CONSOLE.log(f"  [yellow]⚠ Point cloud has no colors[/yellow]")
        
        # 文件大小
        file_size = os.path.getsize(output_file) / (1024 * 1024)  # MB
        CONSOLE.log(f"  File size: {file_size:.2f} MB")
    else:
        CONSOLE.log(f"  [red]✗ Point cloud is empty![/red]")
else:
    CONSOLE.log(f"[red]✗ Output file not found: {output_file}[/red]")

CONSOLE.log("\n[green]Point cloud generation pipeline completed successfully![/green]")
CONSOLE.log(f"  Output: {output_file}")


[19:59:18] ✓ Output file exists: /root/autodl-tmp/nuScenes/preprocess/trainval/000/Drop50/0.ply           ]8;id=598300;file:///tmp/ipykernel_96148/416561032.py\416561032.py]8;;\:]8;id=529507;file:///tmp/ipykernel_96148/416561032.py#3\3]8;;\

[19:59:19] Point cloud statistics:                                                                        ]8;id=360023;file:///tmp/ipykernel_96148/416561032.py\416561032.py]8;;\:]8;id=439475;file:///tmp/ipykernel_96148/416561032.py#8\8]8;;\

             Point count: 5290624                                                                         ]8;id=841764;file:///tmp/ipykernel_96148/416561032.py\416561032.py]8;;\:]8;id=373105;file:///tmp/ipykernel_96148/416561032.py#9\9]8;;\

             X range: [-20.00, 20.00]                                                                    ]8;id=52224;file:///tmp/ipykernel_96148/416561032.py\416561032.py]8;;\:]8;id=532913;file:///tmp/ipykernel_96148/416561032.py#13\13]8;;\

             Y range: [-20.00, 3.07]                                                                     ]8;id=171025;file:///tmp/ipykernel_96148/416561032.py\416561032.py]8;;\:]8;id=630630;file:///tmp/ipykernel_96148/416561032.py#14\14]8;;\

             Z range: [-20.00, 119.99]                                                                   ]8;id=588776;file:///tmp/ipykernel_96148/416561032.py\416561032.py]8;;\:]8;id=362695;file:///tmp/ipykernel_96148/416561032.py#15\15]8;;\

             ✓ Point cloud has colors                                                                    ]8;id=120878;file:///tmp/ipykernel_96148/416561032.py\416561032.py]8;;\:]8;id=493601;file:///tmp/ipykernel_96148/416561032.py#18\18]8;;\

             File size: 136.23 MB                                                                        ]8;id=129261;file:///tmp/ipykernel_96148/416561032.py\416561032.py]8;;\:]8;id=751099;file:///tmp/ipykernel_96148/416561032.py#24\24]8;;\

                                                                                                         ]8;id=488681;file:///tmp/ipykernel_96148/416561032.py\416561032.py]8;;\:]8;id=878310;file:///tmp/ipykernel_96148/416561032.py#30\30]8;;\
           Point cloud generation pipeline completed successfully!                                                      

             Output: /root/autodl-tmp/nuScenes/preprocess/trainval/000/Drop50/0.ply                      ]8;id=39809;file:///tmp/ipykernel_96148/416561032.py\416561032.py]8;;\:]8;id=187137;file:///tmp/ipykernel_96148/416561032.py#31\31]8;;\